**Plotting Expected Value of Transits Over Time**

This code produces a specified number of figures which plot the expected number of transits for a set of systems over time.
To run, call the "run" function at the end of the file, and specify the range of files for which you wish to generate plots.

In [6]:
from generate_lib import *

In [7]:
# Debug flags
SHOW_PLOTS = False
SAVE_PLOTS = True

In [8]:
# Formats the data on the x-axis
def formatter(x, pos):
    return '%1.1f' % (x/1000000.0)

# Plots the data and saves the figure.
def plot_data(data, savename):
    # Initializes the plot
    (fig, ax) = plt.subplots()

    (x, y) = (data['axes']['x'], data['axes']['y'])
    max_planets = data['max_planets']

    # Plots the data
    plt.plot(x, y, label = savename)

    # Labels the plot
    ax.set_title("Multiplicity Expecation Values vs. Time " + "({planets}-Planet System)".format(planets = max_planets), fontsize = 15)
    ax.set_xlabel('Time (Myr)', fontsize = 12)
    ax.set_ylabel('Expectation Value (Dimensionless)', fontsize = 12)

    # Gets the max time on the time axis.
    max_time = np.nanmax(x)

    # Gets the median and mean of the data.
    mean = np.nanmean(y)
    median = np.nanmedian(y)

    # Displays the mean and median lines.
    ax.hlines(mean, 0, max_time, 'orange', label = 'Mean')
    ax.hlines(median, 0, max_time, 'red', label = 'Median')

    # Sets the limits, tick spacing, and label format for the x-axis
    ax.set_xlim(0, max_time)
    ax.xaxis.set_major_locator(tck.LinearLocator(11))
    ax.xaxis.set_major_formatter(tck.FuncFormatter(formatter))

    ax.legend()

    if SHOW_PLOTS:
        plt.show()

    # Saves the figure using a specified path
    if SAVE_PLOTS:
        save_path = os.path.join(os.pardir, os.pardir, 'results', 'multiplicities', 'exp-values', 'ev_' + savename)
        fig.savefig(save_path, bbox_inches="tight", dpi=300)
    
    plt.close()

# Plots the data and saves the figure.
def multiplot_data(system_name, data, rdata):
    # Initializes the plot
    (fig, axes) = plt.subplots(1, 3, sharey = True)
    fig.set_size_inches(15, 5)

    (x, y) = (data['axes']['x'], data['axes']['y'])
    (rx, ry) = (rdata['axes']['x'], rdata['axes']['y'])
    
    data_good = (data['max_planets'] != 0)
    rdata_good = (rdata['max_planets'] != 0)

    max_planets = data['max_planets'] if data_good else (rdata['max_planets'] if rdata_good else None)

    if max_planets == None:
        return

    if data_good and rdata_good:
        max_time = max(np.nanmax(x), np.nanmax(rx))
    else:
        max_time = np.nanmax([x, rx][rdata_good])

    # Plots the data
    plt.subplot(1, 3, 1)

    if data_good:
        plt.scatter(x, y, color = 'red', alpha = 0.3, label = system_name, s = 2)
    
    if rdata_good:
        plt.scatter(rx, ry, color = 'blue', alpha = 0.3, label = system_name + '_randomO', s = 2)

    plt.subplot(1, 3, 2)

    if data_good:
        plt.plot(x, y, color = 'red')

    plt.subplot(1, 3, 3)
    
    if rdata_good:
        plt.plot(rx, ry, color = 'blue')

    for col in range(len(axes)):
        ax = axes[col]
        ax.grid(True, axis = 'y', alpha = 0.5)

        ax.set_xlabel('Time (Myr)', fontsize = 12)

        if col == 0:
            ax.set_ylabel('Expectation Value (Dimensionless)', fontsize = 12)

        ax.set_xlim(0, max_time)
        ax.xaxis.set_major_locator(tck.LinearLocator(11))
        ax.xaxis.set_major_formatter(tck.FuncFormatter(formatter))

        if col == 1:
            mean = np.nanmean(y)
            median = np.nanmedian(y)

            # Displays the mean and median lines.
            ax.hlines(mean, 0, max_time, color = 'orange', label = 'Mean (aligned)', linestyle = 'dashed')
            ax.hlines(median, 0, max_time, color = 'orange', label = 'Median (aligned)')
        elif col == 2:
            mean = np.nanmean(ry)
            median = np.nanmedian(ry)

            # Displays the mean and median lines.
            ax.hlines(mean, 0, max_time, color = 'purple', label = 'Mean (random)', linestyle = 'dashed')
            ax.hlines(median, 0, max_time, color = 'purple', label = 'Median (random)')            
    
    fig.suptitle("Multiplicity Transit Probability Expectation Values vs. Time ({planets}-Planet System)".format(planets = max_planets), fontsize = 17)
    fig.legend(bbox_to_anchor = (0.9, 0.6), loc = 'upper left')

    plt.subplots_adjust(wspace=0.1, hspace=0)

    if SHOW_PLOTS:
        plt.show()

    # Saves the figure using a specified path
    if SAVE_PLOTS:
        save_path = os.path.join(os.pardir, os.pardir, 'results', 'multiplicities', 'exp_values', 'ev_' + system_name)
        fig.savefig(save_path, bbox_inches = "tight", dpi = 300)
    
    plt.close()

In [9]:
# Runs the code over the specified range of files. The index range corresponds to indexes of the
# data_files list, which contains each of the data file names.
def run(systems):
    assert type(systems) == list

    for system in systems:
        suffix = '_randomO'
        system_name = system.replace(suffix, '').replace('.csv', '')

        if suffix in system_name:
            continue

        p = os.path.join(path, system_name + '.csv')
        rp = os.path.join(path, system_name + suffix + '.csv')

        (data, rdata) = (get_data(p), get_data(rp))

        multiplot_data(system_name, data, rdata)

In [10]:
run(data_files[0:30])